In [1]:
import numpy as np
import pandas as pd
import torch
import warnings
from datasets import load_dataset, Dataset
from utils.database import CreateDatabase, SearchDatabase
from utils.model_rag import ModelRag
from transformers import pipeline
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from sentence_transformers import SentenceTransformer

warnings.filterwarnings('ignore')

In [2]:
path_client = 'aep_chat_bot'
collection_name = 'regulatory_documentation'

# Dataset

In [3]:
total = pd.read_excel('data/total.xlsx', dtype={'point': str})
# np_001_15 = np_001_15.rename(columns={'НТД': 'document', 'Пункт': 'point', 'Описание': 'description'})
dataset = Dataset.from_pandas(total)

In [4]:
dataset[0]

{'document': 'НП-005-16',
 'point': '1',
 'description': 'Настоящие федеральные нормы и правила в области использования атомной энергии "Положение о порядке объявления аварийной обстановки, оперативной передачи информации и организации экстренной помощи атомным станциям в случаях радиационно опасных ситуаций" (далее - Положение) разработаны в соответствии со статьей 6 Федерального закона от 21 ноября 1995 г. N 170-ФЗ "Об использовании атомной энергии" (Собрание законодательства Российской Федерации, 1995, N 48, ст. 4552; 1997, N 7, ст. 808; 2001, N 29, ст. 2949; 2002, N 1, ст. 2; N 13, ст. 1180; 2003, N 46, ст. 4436; 2004, N 35, ст. 3607; 2006, N 52, ст. 5498; 2007, N 7, ст. 834; N 49, ст. 6079; 2008, N 29, ст. 3418; N 30, ст. 3616; 2009, N 1, ст. 17; N 52, ст. 6450; 2011, N 29, ст. 4281; N 30, ст. 4590, ст. 4596; N 45, ст. 6333; N 48, ст. 6732; N 49, ст. 7025; 2012, N 26, ст. 3446; 2013, N 27, ст. 3451), постановлением Правительства Российской Федерации от 1 декабря 1997 г. N 1511 "Об

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Create Database

In [6]:
# embedding_model = SentenceTransformer(
#     "intfloat/multilingual-e5-large",
#     model_kwargs={'torch_dtype': torch.float16}
# )

# create_db = CreateDatabase(
#     embedding_model=embedding_model,
#     dataset=dataset,
#     device=device,
#     collection_name=collection_name,
#     path=path_client    
# )

# create_db.create()

# Generation

In [8]:
generation_pipeline = pipeline(
    'text-generation',
    # model='Qwen/Qwen2-1.5B-Instruct',
    model='train_model/Qwen_Qwen2_1.5B_Instruct',
    device=device,
    torch_dtype=torch.float16
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cuda


In [9]:
embedding_model = SentenceTransformer(
    "intfloat/multilingual-e5-large",
    model_kwargs={'torch_dtype': torch.float16}
)

In [10]:
cross_encoder = HuggingFaceCrossEncoder(
    model_name='amberoad/bert-multilingual-passage-reranking-msmarco',
    model_kwargs={'device': device}
)

In [11]:
model = ModelRag(
    generation_pipeline=generation_pipeline,
    embedding_model=embedding_model,
    cross_encoder=cross_encoder,
    path_client=path_client,
    device=device,
    collection_name=collection_name,
    temperature=0.9,
    top_p=0.6,
    max_new_tokens=1024
)

In [12]:
query = 'Опиши четыре класса безопасности по НП-001-15. Расскажи подробно что они включают и какие эелементы к ним относятся? В каком пункте НП-001-15 идет их описание?'
answer, context = model.predict(query)
print(answer)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


По данным НП-001-15, четыре класса безопасности определены следующим образом:

1. Класс 1 - это класс безопасности, который относится к элементам, которые могут стать исходным событием аварии, приводящей к повреждению тепловыделяющего элемента с превышением максимального проектного предела. Элементы этого класса безопасности не играют важной роли в безопасности АС и не влияют на безопасность системы.

2. Класс 2 - этот класс безопасности относится к элементам, которые могут стать исходным событием аварии, приводящей к повреждению тепловыделяющего элемента без превышения максимального проектного предела при проектном функционировании систем безопасности с учетом нормируемого для проектных аварий количества отказов в указанных системах. Эти элементы AС играют ключевую роль в обеспечении безопасности АС.

3. Класс 3 - этот класс безопасности относится к элементам, которые важны для безопасности, но не вошли в классы 1 и 2. Они играют роль в обеспечении безопасности АС, но не играют решающ

In [13]:
query = 'Расскажи про 5 уровней глубокоэшелонированной защиты'
answer, context = model.predict(query)
print(answer)

5 уровней глубокоэшелонированной защиты в атомной энергетике:

1. Уровень 1: Условия размещения и предотвращение отклонений от нормальной эксплуатации. Это включает в себя оценку и выбор подходящего места для размещения атомной станции, установку санитарно-защитной зоны и зоны наблюдения вокруг станции. Также важна работа над защитой барьеров, таких как теплоноситель реактора, герметичное ограждение РУ и биологическую защиту.

2. Уровень 2: Защита от воздействия радиоактивных веществ. Здесь необходимо обеспечивать безопасность от радиоактивных веществ, которые могут попасть в окружающую среду при работе атомной станции.

3. Уровень 3: Защита от воздействия ионизирующего излучения. Этот уровень обеспечивает защиту от воздействия ионизирующего излучения, которое может быть вызвано процессами работы атомной станции.

4. Уровень 4: Защита от воздействия радиационных материалов. Этот уровень обеспечивает защиту от воздействия радиационных материалов, которые могут попасть в окружающую среду

In [14]:
query = 'Перечисли основные функции систем безопасности на атомной станции'
answer, context = model.predict(query)
print(answer)

Основные функции систем безопасности на атомной станции включают:

1. Аварийное остановление реактора и поддержание его в подкритическом состоянии: 
   Эта функция обеспечивает возможность быстрого прекращения работы атомной станции в случае возникновения аварийного события, такого как ошибка в управлении или отказ в системах управления.

2. Аварийное отводование тепла от реактора: 
   Этот процесс помогает поддерживать температуру реактора в безопасном диапазоне даже при возникновении аварийного события.

3. Удержание радиоактивных веществ в установленных границах: 
   Эта функция позволяет контролировать и ограничивать распространение радиации, если произойдет аварийное событие.

4. Предотвращение взаимного влияния систем безопасности: 
   Функции безопасности должны быть отделены друг от друга, чтобы обеспечить эффективное выполнение своих задач.

5. Поддержание подкритической ситуации при хранении и транспортировании ядерного топлива: 
   Эта функция помогает контролировать и огран

In [22]:
query = 'Какие группы оборудования и трубопроводов по НП-089 существуют на атомной станции?'
answer, context = model.predict(query)
print(answer)

На атомной станции могут существовать различные группы оборудования и трубопроводов, в зависимости от их функций и характеристик. Однако, без конкретного списка всех групп, которые могут быть использованы на атомной станции, сложно предоставить точный ответ. 

В общем случае, на атомной станции может быть установлено оборудование и трубопроводы, которые:

1. Имеют КИП для измерения давления, температуры, расхода, уровня рабочей среды, химического состава теплоносителя и контроля перемещений.
2. Устройства для отбора среды.
3. Датчики перемещения оборудования и трубопроводов.
4. Разработчики проектов реакторной установки (РУ) и АЭУ определяют принадлежность каждого оборудования и трубопровода к группе.
5. Направление работы оборудования и трубопроводов может быть вакуумметрическим, избыточным, гидростатическим или вакуумным.

Однако, чтобы получить более точный ответ, вам следует обратиться к специалистам в области атомной энергетики или к соответствующим организациям. Они смогут предос